In [3]:
import pandas as pd

In [4]:
dad = pd.read_csv('data/H_DAD_w1a3.csv')

In [5]:
with_cdr = dad.loc[pd.notna(dad['r1cdr_final'])]
with_cdr['r1cdr_final'].value_counts()

0.5    1568
0.0     768
1.0     162
2.0      25
3.0       5
Name: r1cdr_final, dtype: int64

In [8]:
sample_cols = '''
r1agey
ragender
raeduc_l
raeducl
r1illiterate
h1rural
r1hmse_score
r1word_total
r1wre_score
r1log_reco
r1bm_immex
r1bm_reclex
r1verbal
r1csid_score
r1rv_score
r1jormscore
r1bl1score
r1bl2score
r1act_tv
r1act_read
r1act_chor
r1act_comp
r1act_nap
r1act_meal
r1act_trav
r1act_pubt
r1act_work
r1act_stor
r1act_walk
r1act_spor
r1act_dail
r1csi1
r1csi2
r1csi3
r1csi4
r1csi5
r1csi6
r1csi7
r1csi8
r1csi9
r1csi10
r1csi11
r1csi12
r1csi13
r1csi14
r1csi15
r1i_hear
r1i_sleep
r1i_memory
r1i_compmem
r1i_menabil
r1i_compabil
r1systo
r1diasto
r1bphigh
r1bmicat
r1cesd10'''.split()

In [9]:
from sklearn.model_selection import train_test_split

cdr_cols = [x for x in with_cdr.columns if 'cdr' in x]
# X = with_cdr[with_cdr.columns[4:-1]]
# X = with_cdr[with_cdr.columns[~with_cdr.isna().any()][4:]]
X = with_cdr[sample_cols]
y = with_cdr['r1cdr_final']
X = X[X.columns[~X.isna().any()]]  # drop features with missing
X = X[[x for x in X.columns if x not in cdr_cols]]

y = pd.to_numeric(y)
y.loc[y < 1] = 0
y.loc[y > 0] = 1
seed = 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed, stratify=y)

In [10]:
from interpret.glassbox import ExplainableBoostingClassifier
ebm = ExplainableBoostingClassifier(random_state=seed)
ebm.fit(X_train, y_train)

ExplainableBoostingClassifier(feature_names=['r1agey', 'ragender', 'raeduc_l',
                                             'raeducl', 'r1illiterate',
                                             'h1rural', 'r1hmse_score',
                                             'r1word_total', 'r1wre_score',
                                             'r1log_reco', 'r1bm_immex',
                                             'r1bm_reclex', 'r1verbal',
                                             'r1csid_score', 'r1rv_score',
                                             'r1jormscore', 'r1bl2score',
                                             'r1act_tv', 'r1act_read',
                                             'r1act_chor', 'r1act_comp',
                                             'r1act_nap', 'r1act_meal',
                                             'r1act_trav', '...
                                             'categorical', 'categorical',
                                             'contin

In [12]:
from interpret.perf import ROC
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
from interpret import show
ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='ebm')
show(ebm_perf)

<!-- http://127.0.0.1:7001/140179800567376/ -->

/home/jon/miniconda3/envs/neurohack/lib/python3.9/site-packages/interpret/visual/udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/home/jon/miniconda3/envs/neurohack/lib/python3.9/site-packages/interpret/visual/udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/home/jon/miniconda3/envs/neurohack/lib/python3.9/site-packages/interpret/visual/udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


In [15]:
set_visualize_provider(InlineProvider())
show(ebm.explain_global())
# ebm_local = ebm.explain_local(X_test[:20], y_test[:20])
# show(ebm_local)